In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from coherenceModel import *
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [2]:
paragraph_df = pd.read_csv('LATimesWashPostPerms.csv')
paragraph_df

,paragraph,is_coherent,id,num_sent
0,A new supercomputer model of the universe...,1,0,12
1,1. ``Mrs. Doubtfire'' 2. ``A Perfect ...,1,1,11
2,Five key U.S. maritime unions formally ag...,1,2,12
3,OUR HOURLY BREAD: Bread doesn't demand mu...,1,3,12
4,"LYON, France &MD Much of the criticism ...",1,4,12
...,...,...,...,...
41995,"Before he can leave the United States, Abu Mar...",0,1999,11
41996,State Department spokesman Nicholas Burns said...,0,1999,11
41997,``The decision was taken on the basis of overa...,0,1999,11
41998,An INS spokesman said the agency's case to exc...,0,1999,11


In [3]:
train_pars = paragraph_df[paragraph_df['id'] < 1440]
dev_pars = paragraph_df[(paragraph_df['id'] >= 1440) & (paragraph_df['id'] < 1800)]
test_pars = paragraph_df[paragraph_df['id'] >= 1800]

def get_balance(df):
    print(len(df[df['is_coherent'] == 1]) / len(df))

X_train, y_train = train_pars.paragraph.values, train_pars.is_coherent.values
X_val, y_val = dev_pars.paragraph.values, dev_pars.is_coherent.values
X_test, y_test = test_pars.paragraph.values, test_pars.is_coherent.values

get_balance(train_pars)
get_balance(dev_pars)
get_balance(test_pars)

print(X_train[0])
print(y_train[0])

0.047619047619047616
0.047619047619047616
0.047619047619047616
     A new supercomputer model of the universe  &MD  the most complex such simulation ever created  &MD  lends credence to the theory that the original recipe for the real cosmos probably included both cold and hot versions of the mysterious ingredient known as dark matter. An astrophysics team at the University of Illinois used a massively parallel supercomputer to visualize the universe as it would be seen in X-rays, which are emitted by superhot gases or in violent events. ``Our simulation is the first that is sufficiently comprehensive to make theoretical predictions that can be compared with observations,'' said team leader Michael Norman, who presented the work at a meeting of the American Astronomical Society last week in Minneapolis. The new model agrees so well with actual observations, he said, that it may guide astronomers toward new discoveries. In the last decade, many astronomers came to believe that at least 

In [4]:
import gensim.downloader
embed = gensim.downloader.load("glove-wiki-gigaword-50")

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [8]:
################ SET THIS TO CHANGE WINDOW SIZE OF THINGS BELOW:
wsize = 5
################
train_data = WindowedParDataset(X_train, y_train, embed, wsize)
dev_data = WindowedParDataset(X_val, y_val, embed, wsize)
test_data = WindowedParDataset(X_test, y_test, embed, wsize)
train_loader = DataLoader(train_data, batch_size=25, collate_fn=basic_collate_fn, shuffle=True)
dev_loader = DataLoader(dev_data, batch_size=25, collate_fn=basic_collate_fn, shuffle=True)
test_loader = DataLoader(test_data, batch_size=25, collate_fn=basic_collate_fn, shuffle=False)

Complete
Complete
Complete


In [9]:
num_pos = len(paragraph_df[paragraph_df["is_coherent"] == 1])
num_neg = len(paragraph_df[paragraph_df["is_coherent"] == 0])
dampen = 1
pos_weight = torch.Tensor([num_neg / num_pos / dampen]).to(device)
pos_weight

tensor([20.], device='cuda:0')

In [10]:
# test on validation to see if overfit is possible
ffnn = FFNN(5, device)
ffnn.to(device)
optim = get_optimizer(ffnn, lr=1e-2, weight_decay=0)
best_model, stats = train_model(ffnn, dev_loader, dev_loader, optim, pos_weight=pos_weight,
                                num_epoch=15, collect_cycle=20, device=device, patience=None)
plot_loss(stats)

------------------------ Start Training ------------------------
Epoch No. 1--Iteration No. 2181-- batch loss = 0.6201
Validation UAR: 0.5384
Validation accuracy: 0.8352
Validation loss: 1.3365
Epoch No. 2--Iteration No. 4362-- batch loss = 1.2954
Validation UAR: 0.5574
Validation accuracy: 0.4538
Validation loss: 1.3217
Epoch No. 3--Iteration No. 6543-- batch loss = 1.8060
Validation UAR: 0.5663
Validation accuracy: 0.4141
Validation loss: 1.3144
Epoch No. 4--Iteration No. 8724-- batch loss = 0.9277
Validation UAR: 0.5802
Validation accuracy: 0.6283
Validation loss: 1.3045
Epoch No. 5--Iteration No. 10905-- batch loss = 2.2346
Validation UAR: 0.5878
Validation accuracy: 0.5773
Validation loss: 1.2957
Epoch No. 6--Iteration No. 13086-- batch loss = 2.1687
Validation UAR: 0.5942
Validation accuracy: 0.5421
Validation loss: 1.2874
Epoch No. 7--Iteration No. 15267-- batch loss = 0.7313
Validation UAR: 0.5982
Validation accuracy: 0.4892
Validation loss: 1.2803
Epoch No. 8--Iteration No. 17

KeyboardInterrupt: 

In [ ]:
import itertools
from tqdm.notebook import tqdm

torch.cuda.empty_cache()

def search_param_utterance(wsize):
    """Experiemnt on different hyper parameters."""
    learning_rate, weight_decay = get_hyper_parameters()
    window_sizes = [wsize]
    print("learning rate from: {}\nweight_decay from: {}\nwindow from: {}".format(
        learning_rate, weight_decay, window_sizes
    ))
    best_model, best_stats = None, None
    best_accuracy, best_lr, best_wd, best_window_size = 0, 0, 0, 0
    for lr, wd, window_size in tqdm(itertools.product(learning_rate, weight_decay, window_sizes),
                           total=len(learning_rate) * len(weight_decay) * len(window_sizes)):
        net = FFNN(window_size, device).to(device)
        optim = get_optimizer(net, lr=lr, weight_decay=wd)
        model, stats = train_model(net, train_loader, dev_loader, optim, pos_weight=pos_weight, 
                                   num_epoch=100, collect_cycle=500, device=device, 
                                   verbose=True, patience=5, stopping_criteria='accuracy')
        # print accuracy
        print(f"{(lr, wd, window_size)}: {stats['accuracy']}")
        # update best parameters if needed
        if stats['accuracy'] > best_accuracy:
            best_accuracy = stats['accuracy']
            best_model, best_stats = model, stats
            best_lr, best_wd, best_window_size = lr, wd, window_size
            torch.save(best_model.state_dict(), 'best_rnn_latimes.pt')
    print("\n\nBest learning rate: {}, best weight_decay: {}, best window: {}".format(
        best_lr, best_wd, best_window_size))
    print("Accuracy: {:.4f}".format(best_accuracy))
    plot_loss(best_stats)
    return best_model
basic_model = search_param_utterance(wsize)

learning rate from: [0.01]
weight_decay from: [0.0002, 0.002, 0.005, 0.01]
window from: [5]


  0%|          | 0/4 [00:00<?, ?it/s]

------------------------ Start Training ------------------------
Epoch No. 1--Iteration No. 8646-- batch loss = 2.0668
Validation UAR: 0.5278
Validation accuracy: 0.3419
Validation loss: 1.3305
Epoch No. 2--Iteration No. 17292-- batch loss = 1.3664
Validation UAR: 0.6198
Validation accuracy: 0.3861
Validation loss: 1.2688
Epoch No. 3--Iteration No. 25938-- batch loss = 0.6299
Validation UAR: 0.6084
Validation accuracy: 0.4007
Validation loss: 1.2038
Epoch No. 4--Iteration No. 34584-- batch loss = 0.6006
Validation UAR: 0.6192
Validation accuracy: 0.4590
Validation loss: 1.2020
Epoch No. 5--Iteration No. 43230-- batch loss = 1.2345
Validation UAR: 0.6220
Validation accuracy: 0.3635
Validation loss: 1.1849


In [11]:
basic_model = FFNN(5, device)
basic_model.load_state_dict(torch.load('best_rnn.pt'))
basic_model.eval()
basic_model.to(device)
uar, accuracy, total_loss = get_validation_performance(
    basic_model, 
    nn.BCEWithLogitsLoss(pos_weight=pos_weight), 
    test_loader, 
    device
)
print("Final selection: window size 5 with Q = 0.0002")
print("Test UAR: {:.4f}".format(uar))
print("Test accuracy: {:.4f}".format(accuracy))
print("Test loss: {:.4f}".format(total_loss))

Final selection: window size 5 with Q = 0.0002
Test UAR: 0.7660
Test accuracy: 0.7768
Test loss: 0.9321
